In [1]:
import os 
import pandas as pd
import json
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize, sent_tokenize
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession

/Users/lam.nguyen/Desktop/GithubClone/BigData_Final/.venv/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/lam.nguyen/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lam.nguyen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lam.nguyen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Preprocessing

In [3]:
# def merge_files() -> None:
#     """Merge all JSON files of a folder

#     Sample folder structure
#     ```
#     └── 📁data
#         └── 📁merged
#             └── test.json
#         └── 📁processed
#         └── 📁raw
#             └── test01.json
#             └── test02.json
#     ```
#     Args:
#         None
#     Return:
#         returns None"""
#     files = os.listdir(os.path.join("..", "data", "raw"))
#     text_todf = []
#     for i in range(len(files)):
#         with open(os.path.join("..", "data", "raw", files[i]), 'r') as file:
#             data = json.load(file)
#             df_metadata = pd.json_normalize(data, max_level=1)
#             text = ""
#             for element in df_metadata['body_text'][0]:
#                 text += element['text']
#             text_todf.append(text)
#     df = pd.DataFrame({"text": text_todf})
#     df.to_json(os.path.join("..", "data", "merged", "corpus.json"))

In [38]:
# Read data
data = pd.read_json(os.path.join("..", "data", "merged", "test.json"))

In [5]:
def custom_word_tokenize(sent_tokens: list[str]) -> list[str]:
    """Tokenises words 
    
    Args:
        sent_tokens: tokens of sentences
    Returns:
        A list of word tokens"""
    word_tokens = []
    for sent in sent_tokens:
        tokens = word_tokenize(sent)
        word_tokens.extend(tokens)
    return word_tokens

In [41]:
def preprocess(data: pd.DataFrame) -> pd.DataFrame:
    """Performs the preprocessing
    
    Args:
        data: input file as pandas.DataFrame
    Returns:
        A pandas.DataFrame which as vector form of the input
    """
    stop_words = stopwords.words('english')
    lemmatiser = WordNetLemmatizer()

    #Sentence tokenisation
    data['sent_tokens'] = data['text'].apply(sent_tokenize)
    
    # Text cleaning
    data['sent_tokens'] = data['sent_tokens'].apply(lambda sentences: [re.sub(r"[^a-zA-Z\s]", "", sent) for sent in sentences])
    
    # # Normalisation
    data['sent_tokens'] = data['sent_tokens'].apply(lambda sentences: [sent.lower() for sent in sentences])

    #Word tokenisation
    data['word_tokens'] = data['sent_tokens'].apply(custom_word_tokenize)

    # # Stemming
    data['word_tokens'] = data['word_tokens'].apply(lambda word_tokens: [lemmatiser.lemmatize(word) for word in word_tokens])

    # Word Embeddings (turns into vector)
    data['to_tfidf'] = data['word_tokens'].apply(lambda tokens: ' '.join(tokens))
    vectorizer = TfidfVectorizer(min_df=0.3, max_df=0.85, stop_words=stop_words)
    tfidf_matrix = vectorizer.fit_transform(data['to_tfidf'])
    
    # Convert to DataFrame
    tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
    return data, tfidf

In [42]:
data, tfidf  = preprocess(data)

# Pyspark LDA

In [43]:
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession

In [10]:
# Create spark spession
spark = SparkSession.builder\
        .appName("Nhom09_PySparkLDA")\
        .master("local[*]")\
        .config("spark.driver.bindAddress", "localhost")\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/28 08:49:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# Fit tfidf (panads.DataFrame) into data structure of spark
tfidf_ps = spark.createDataFrame(tfidf)
tfidf_ps = tfidf_ps.rdd.map(lambda row: (Vectors.dense(row), )).toDF(["features"])


In [45]:
from pyspark.ml.clustering import LDA

# Create an LDA model
lda = LDA(k=5, maxIter=10)  # k is the number of topics

# Fit the LDA model
lda_model = lda.fit(tfidf_ps)
# Describe topics
topics = lda_model.describeTopics(3)  # Get the top 3 terms for each topic
topics.show(truncate=False)

+-----+---------------+---------------------------------------------------------------------+
|topic|termIndices    |termWeights                                                          |
+-----+---------------+---------------------------------------------------------------------+
|0    |[404, 41, 105] |[0.002398466433135447, 0.002334772230784771, 0.0023063777294193063]  |
|1    |[343, 164, 35] |[0.0023859780701145855, 0.0023294478680108786, 0.0023203198160067742]|
|2    |[108, 8, 232]  |[0.0024689895728190795, 0.002377024731400353, 0.0023564226010567954] |
|3    |[176, 220, 273]|[0.0023427878938911107, 0.0023394471897149522, 0.0023343600894544132]|
|4    |[26, 196, 278] |[0.0024366442857787266, 0.002359836168195985, 0.002356088139912443]  |
+-----+---------------+---------------------------------------------------------------------+

